In [8]:
import lancedb
import pandas as pd
import pyarrow as pa

uri = "data/sample-lancedb"
db = lancedb.connect(uri)

In [9]:
if "my_table" in db.table_names():
    db.drop_table("my_table")

db.create_table(
    "my_table",
    schema=pa.schema(
        [
            ("id", pa.int32()),
            ("value", pa.float32()),
            ("vector", pa.list_(pa.float32(), 3)),
        ]
    ),
)
my_table = db.open_table("my_table")

In [16]:
my_table.add(
    pd.DataFrame(
        {
            "id": [1, 2, 3],
            "value": [0.1, 0.2, 0.3],
            "vector": [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]],
        }
    ),
    mode="overwrite"
)
print(my_table.to_pandas())

   id  value           vector
0   1    0.1  [0.1, 0.2, 0.3]
1   2    0.2  [0.4, 0.5, 0.6]
2   3    0.3  [0.7, 0.8, 0.9]


In [19]:
# Query by vector
query_vector = [0.6, 0.9, 0.3]

results = my_table.search(query_vector, "vector").limit(2).to_pandas()
print(results)

   id  value           vector  _distance
0   2    0.2  [0.4, 0.5, 0.6]       0.29
1   3    0.3  [0.7, 0.8, 0.9]       0.38
